In [ ]:
import os
import sys
from pprint import pprint as pp
from time import time as tt

import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as plt3d
import matplotlib.colors
# from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import pandas as pd
import seaborn as sns
import torch
print("PyTorch version:",print(torch.__version__),", CUDA version:", torch.version.cuda)
# from torch_geometric.data import Data
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch_scatter import scatter_add
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import clear_output
from IPython.display import HTML, display

%matplotlib inline

# Get rid of RuntimeWarnings, gross
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import numpy as np
import pandas as pd

dtype = torch.float
device = torch.device("cuda")



In [ ]:
layers_x = [-999, -21, 151, 497, 843.7, 1141, 1266, 1960]
layers_y = [-16, 131, 477, 832, 1146, 1479.6, 1973 ]
layers_ordered = [-16, -21, 131, 151, 477, 497, 832, 832, 843.7, 843.7, 1146, 1146, 1141, 1141, 1252.7, 1252.7, 1266, 1266, 1479.6, 1479.6, 1467.4, 1467.4, 1973, 1973, 1960, 1960]

In [ ]:
# Read hits
def readMuonE(event_file):
   data = pd.read_csv(event_file) 
   
   iEventLast=data.iat[0,0]
   k=0
   dataOUT = []

   #iterate over input data
   for k, row in data.iterrows():
     e = int(row[0])
     if len(dataOUT) < e:
       dataOUT.append([])
     dataOUT[e-1].extend([row[1], row[3]])

   return dataOUT       

# Read slopes of MC tracks
def readMCslopes(event_file):  
  data = pd.read_csv(event_file) 
  
  temp=data.iat[-1,0] # number of events
  track_slopes_x=torch.zeros(temp,6)
  track_slopes_y=torch.zeros(temp,6)

  axis_oi=[4,1]
  for k, row in data.iterrows():
    if row[7] in (0,1,2):
      if track_slopes_x.shape[0]< row[0]-1 :
        track_slopes_x=torch.cat([track_slopes_x,torch.zeros(1,6)],dim=0)
      track_slopes_x[int(row[0])-1,int(row[7]*2) ] = row[axis_oi[0]]/row[6]
      track_slopes_x[int(row[0])-1,int(row[7])*2+1] =  row[axis_oi[1]] - row[axis_oi[0]]/row[6]  * row[3]
  
  axis_oi=[5,2]
  for k, row in data.iterrows():
    if row[7] in (0,1,2):
      if track_slopes_y.shape[0]< row[0]-1 :
        track_slopes_y=torch.cat([track_slopes_y,torch.zeros(1,6)],dim=0)
      track_slopes_y[int(row[0])-1,int(row[7]*2) ] = row[axis_oi[0]]/row[6]
      track_slopes_y[int(row[0])-1,int(row[7])*2+1] =  row[axis_oi[1]] - row[axis_oi[0]]/row[6]  * row[3]

  return np.concatenate((track_slopes_x, track_slopes_y), axis=1)


In [ ]:
# Read data and prepare for training

event_file_MC= "path\\to\\MCtrackFile_big.csv"
event_file= "path\\to\\hitFile_big.csv"
event_file_recon= "path\\to\\trackFile_big.csv"

import time

start_time = time.time()
hit = readMuonE(event_file)
print("readMuonE\t--- %s seconds ---" % (time.time() - start_time))
print(len(hit))

start_time = time.time()
mc = readMCslopes(event_file_MC)
print("readMCtrack\t--- %s seconds ---" % (time.time() - start_time))
print(len(mc))

# split data into training and test datasets
train_dataset_,test_dataset_,test_tracks,train_tracks,train_recon,test_recon, train_mc_, test_mc_ = [],[],[],[],[],[],[],[]
for k in range(len(hit)):
  # skip not full-length events
  if len(hit[k]) != 52 or len(mc[k]) != 12:
      continue
  if (k%4 == 0):
    test_dataset_.append(hit[k])
    test_mc_.append(mc[k])
  else:
    train_dataset_.append(hit[k])
    train_mc_.append(mc[k])
print(len(train_dataset_))    


In [ ]:
# feature scaling

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, MaxAbsScaler
import sklearn.preprocessing

scaler = StandardScaler()
scaler.fit(train_dataset_)
train_dataset = scaler.transform(train_dataset_)

scaler.fit(test_dataset_)
test_dataset = scaler.transform(test_dataset_)

train_mc = train_mc_
test_mc = test_mc_


In [ ]:
# Turn datasets into tensors
train_dataset = torch.Tensor(train_dataset).cuda()
train_dataset.requires_grad_(True)
train_mc = torch.Tensor(train_mc).cuda()
train_mc.requires_grad_(True)

test_dataset = torch.Tensor(test_dataset)
test_mc = torch.Tensor(test_mc)

In [ ]:

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(52, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 500)
        # self.fc32 = nn.Linear(500, 500)
        # self.fc33 = nn.Linear(500, 500)
        self.fc4 = nn.Linear(500, 500)
        self.fc5 = nn.Linear(500, 12)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        # x = F.relu(self.fc32(x))
        # x = F.relu(self.fc33(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x


In [ ]:
# traint he model

losses = []
model = Model()
model.cuda()
crit = torch.nn.MSELoss().cuda() #nn.L1Loss().cuda()
# crit = torch.nn.L1Loss().cuda()
optimizer =  optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3, amsgrad=True)
# optimizer = torch.optim.SGD(net.parameters(), lr=1e-5, momentum=0.3)

model.train()

for epoch in range(10001):  # loop over the dataset multiple times
    outputs = model(train_dataset)
    loss = crit(outputs, train_mc)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch%100==0:
        print('epoch {}\tloss {}'.format(epoch, loss))
        losses.append([epoch, loss])

print('Finished Training')

In [ ]:
# draw slopes
def plotxy(pred, mc, hits):
    z = np.linspace(-21, 2000, endpoint=True)

    mc1 = mc[2]*z+mc[3]
    mc2 = mc[4]*z+mc[5]
    mc3 = mc[8]*z+mc[9]
    mc4 = mc[10]*z+mc[11]

    pred1 = pred[2]*z+pred[3]
    pred2 = pred[4]*z+pred[5]
    pred3 = pred[8]*z+pred[9]
    pred4 = pred[10]*z+pred[11]

    fig=plt.figure(figsize=(20,10))

    plt.subplot(1,2,1)
    plt.plot(z, mc1, 'k', alpha=0.4)
    plt.plot(z, mc2, 'k', alpha=0.4)
    plt.plot(z, pred1, 'r')
    plt.plot(z, pred2, 'g')
    for i in range(int(len(hits)/2)):
        if hits[2*i+1] in layers_x:
            plt.scatter(hits[2*i+1], hits[2*i], c='k')

    plt.subplot(1,2,2)
    plt.plot(z, mc3, 'k', alpha=0.4)
    plt.plot(z, mc4, 'k', alpha=0.4)
    plt.plot(z, pred3, 'r')
    plt.plot(z, pred4, 'g')
    for i in range(int(len(hits)/2)):
        if hits[2*i+1] in layers_y:
            plt.scatter(hits[2*i+1], hits[2*i], c='k')


In [ ]:
#plot some results
model.eval()
model.cpu()
for i in range(0,20):
    data = test_dataset[i]
    preds=model(data).detach().numpy()
    plotxy(preds, test_mc[i], test_dataset_[i])